In [ ]:
!pip install requests
!pip install pandas
!pip install mp-api
!pip install requests-html

In [12]:
import csv
import os
import pandas as pd
#from ast import literal_eval
import math
import random
#import matplotlib.pyplot as plt
import seaborn as sb
import subprocess

In [13]:
ROOT_DIR = os.path.abspath("")
# DATA_DIR = os.path.join(ROOT_DIR, 'CrabNet/data/materials_data')
DATA_DIR = os.path.join(ROOT_DIR, 'CrabNet/data/materials_data')
TRAINING_CSV_FILE = os.path.join(ROOT_DIR, "Vacancyenergydata.csv")
API_KEY = "3XX6x9Nd3cKWf9Q4wpaio64CBHFp2uDq"
TRAINING_DATA = []

print(TRAINING_CSV_FILE)

/home/murimikelvin/ML-perovskites-project/Vacancyenergydata.csv


## Step1:Collecting data.

#### Load Training data.

In [14]:
df = pd.read_csv(TRAINING_CSV_FILE)
print(df.columns)
df = df[['Chemical formula', 'Vacancy energy [eV/O atom]']]
new_names = {
    'Chemical formula': 'chem_formula',
    'Vacancy energy [eV/O atom]': 'e_ovac'
}
df.rename(columns=new_names, inplace=True)
df = df.to_dict("records")
df[:10]


Index(['Chemical formula', 'A', 'B', 'In literature', 'Valence A', 'Valence B',
       'Radius A [ang]', 'Radius B [ang]', 'Lowest distortion',
       'Formation energy [eV/atom]', 'Stability [eV/atom]',
       'Magnetic moment [mu_B]', 'Volume per atom [A^3/atom]', 'Band gap [eV]',
       'a [ang]', 'b [ang]', 'c [ang]', 'alpha [deg]', 'beta [deg]',
       'gamma [deg]', 'Vacancy energy [eV/O atom]'],
      dtype='object')


[{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'},
 {'chem_formula': 'AcAgO3', 'e_ovac': '0.817'},
 {'chem_formula': 'AcAlO3', 'e_ovac': '6.695'},
 {'chem_formula': 'AcAsO3', 'e_ovac': '3.634'},
 {'chem_formula': 'AcAuO3', 'e_ovac': '0.807'},
 {'chem_formula': 'AcBO3', 'e_ovac': '4.163'},
 {'chem_formula': 'AcBaO3', 'e_ovac': '-1.771'},
 {'chem_formula': 'AcBeO3', 'e_ovac': '-0.080'},
 {'chem_formula': 'AcBiO3', 'e_ovac': '2.653'},
 {'chem_formula': 'AcCaO3', 'e_ovac': '-1.025'}]

## Step 2: Data cleaning

#### Cleaning training data

In [15]:
# Splitting the data into training and predicting and saving this data.
TRAINING_DATA = [m for m in df if m['e_ovac'] != '-']
PREDICTION_DATA = [m for m in df if m['e_ovac'] == '-']



print("Materials in the csv file without E_Ovac: ", len(PREDICTION_DATA))
print("Materials in the csv file with E_Ovac: ", len(TRAINING_DATA))
print("Total materials in the csv file: ", len(df))


Materials in the csv file without E_Ovac:  415
Materials in the csv file with E_Ovac:  4914
Total materials in the csv file:  5329


In [16]:
print(TRAINING_DATA[0])

{'chem_formula': 'Ac2O3', 'e_ovac': '3.150'}


In [17]:
print(PREDICTION_DATA[0])

{'chem_formula': 'AcMgO3', 'e_ovac': '-'}


In [18]:
print(min([x.get('e_ovac') for x in TRAINING_DATA]))
print(max([x.get('e_ovac') for x in TRAINING_DATA]))

-0.006
6.962


#### Create Model Training Data (Create train.csv, val.csv, test.csv)

In [19]:
## Split the data into train, test, val

random.shuffle(TRAINING_DATA) # Shuffle the training data list on every run
split_ratio = "6:2:2" # Set the ratio at with to split the data into train, test and validation sets from the training data list

ratios = [round(int(r)/10 * len(TRAINING_DATA)) for r in split_ratio.split(':')] # Get the number of materials from the training data list according tp the ratio

# Get X amount of data from the training dataset accoring to the set rratio
train_set = TRAINING_DATA[:ratios[0]] 
test_set = TRAINING_DATA[ratios[0]:(ratios[0] + ratios[1])]
val_set = TRAINING_DATA[(ratios[0] + ratios[1]):]

print(len(train_set), len(test_set), len(val_set))
print(ratios, sum(ratios), len(TRAINING_DATA))

print(train_set[:5])
print(test_set[:5])
print(val_set[:5])

2948 983 983
[2948, 983, 983] 4914 4914
[{'chem_formula': 'TbGaO3', 'e_ovac': '4.521'}, {'chem_formula': 'LuBiO3', 'e_ovac': '0.107'}, {'chem_formula': 'ReTcO3', 'e_ovac': '-2.294'}, {'chem_formula': 'SiZrO3', 'e_ovac': '3.899'}, {'chem_formula': 'YbPtO3', 'e_ovac': '-0.622'}]
[{'chem_formula': 'ZnMnO3', 'e_ovac': '-0.703'}, {'chem_formula': 'DyCdO3', 'e_ovac': '-1.035'}, {'chem_formula': 'PmTcO3', 'e_ovac': '1.999'}, {'chem_formula': 'UTmO3', 'e_ovac': '3.505'}, {'chem_formula': 'MnTlO3', 'e_ovac': '-2.514'}]
[{'chem_formula': 'GeBaO3', 'e_ovac': '-7.410'}, {'chem_formula': 'OsNiO3', 'e_ovac': '-4.568'}, {'chem_formula': 'NbSbO3', 'e_ovac': '0.340'}, {'chem_formula': 'NbOsO3', 'e_ovac': '-1.774'}, {'chem_formula': 'GaSrO3', 'e_ovac': '-6.999'}]


In [20]:
## Save data to CSV

fields = ['formula', 'target']
train_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in train_set]
val_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in val_set]
test_rows = [[data.get('chem_formula'), data.get('e_ovac')] for data in test_set]

# print(rows)
## Create train.csv with train_set data
with open(os.path.join(DATA_DIR, 'train.csv'), 'w') as train_file:
    csvwriter = csv.writer(train_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(train_rows)

## Create val.csv with train_set data
with open(os.path.join(DATA_DIR, 'val.csv'), 'w') as val_file:
    csvwriter = csv.writer(val_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(val_rows)

## Create test.csv with train_set data
with open(os.path.join(DATA_DIR, 'test.csv'), 'w') as test_file:
    csvwriter = csv.writer(test_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(test_rows)
    

#### Train the crabnet model

In [23]:



os.chdir ('CrabNet')
command = f"python train_crabnet.py"

process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
for line in process.stdout:
    print(line.decode("utf-8"), end='')


Property "example_materials_property" selected for training

Model architecture: out_dims, d_model, N, heads
3, 512, 3, 4
Running on compute device: cpu
Model size: 11987206 parameters

loading data with up to 3 elements in the formula
training with batchsize 256 (2**8.000)
loading data with up to 3 elements in the formula
stepping every 12 training passes, cycling lr every 1 epochs
checkin at 2 epochs to match lr scheduler
Epoch: 0/40 --- train mae: 3.31 val mae: 3.13
Epoch: 1/40 --- train mae: 3.3 val mae: 3.12
Epoch: 3/40 --- train mae: 3.34 val mae: 3.17
Epoch: 5/40 --- train mae: 3.11 val mae: 2.95
Epoch: 7/40 --- train mae: 3.08 val mae: 2.93
Epoch: 9/40 --- train mae: 3.05 val mae: 2.95
Epoch: 11/40 --- train mae: 2.98 val mae: 2.88
Epoch: 13/40 --- train mae: 2.95 val mae: 2.85
Epoch: 15/40 --- train mae: 2.91 val mae: 2.82
Epoch: 17/40 --- train mae: 2.9 val mae: 2.82
Epoch: 19/40 --- train mae: 2.86 val mae: 2.81
Epoch: 21/40 --- train mae: 2.84 val mae: 2.83
Epoch: 23/40 ---

#### Analyzing training performance

In [ ]:

csv_df = pd.read_csv('model_predictions/example_materials_property_train_output.csv')

ax = sb.regplot(x='target', y='pred-0', data=csv_df)
# ax = sb.scatterplot(x="test", y="result", data=df, palette=color_dict, legend='full')
ax.set_title(f"A regression scatter plot showing the relationship between the predicted and actual oxygen vacancy values after training. ({len(csv_df)} points)")
ax.set_xlabel("Experimental Eovac")
ax.set_ylabel("Predicted Eovac")


#### Create Prediction Data (Create train.csv, val.csv, test.csv)

In [ ]:
## Split the data into train, test, val

random.shuffle(PREDICTION_DATA) # Shuffle the training data list on every run
split_ratio = "6:2:2" # Set the ratio at with to split the data into train, test and validation sets from the training data list

ratios = [round(int(r)/10 * len(PREDICTION_DATA)) for r in split_ratio.split(':')] # Get the number of materials from the training data list according tp the ratio

# Get X amount of data from the training dataset accoring to the set rratio
train_set = PREDICTION_DATA[:ratios[0]] 
test_set = PREDICTION_DATA[ratios[0]:(ratios[0] + ratios[1])]
val_set = PREDICTION_DATA[(ratios[0] + ratios[1]):]

print(len(train_set), len(test_set), len(val_set))
print(ratios, sum(ratios), len(PREDICTION_DATA))

print(train_set[:5])
print(test_set[:5])
print(val_set[:5])

In [ ]:
## Save data to CSV

fields = ['formula', 'target']
train_rows = [[data.get('chem_formula'), random.randint(-1, 1)] for data in train_set]
val_rows = [[data.get('chem_formula'), random.randint(-1, 1)] for data in val_set]
test_rows = [[data.get('chem_formula'), random.randint(-1, 1)] for data in test_set]

# print(rows)
## Create train.csv with train_set data
with open(os.path.join(DATA_DIR, 'train.csv'), 'w') as train_file:
    csvwriter = csv.writer(train_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(train_rows)

## Create val.csv with train_set data
with open(os.path.join(DATA_DIR, 'val.csv'), 'w') as val_file:
    csvwriter = csv.writer(val_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(val_rows)

## Create test.csv with train_set data
with open(os.path.join(DATA_DIR, 'test.csv'), 'w') as test_file:
    csvwriter = csv.writer(test_file)
    
    csvwriter.writerow(fields)
    csvwriter.writerows(test_rows)
    

In [ ]:
print(min([x.get('e_ovac') for x in TRAINING_DATA]))

#### Predicting EOvac values 

In [ ]:
command = f"python predict_crabnet.py.py"

process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
for line in process.stdout:
    print(line.decode("utf-8"), end='')

#### Analyze predicted values

In [ ]:
## Read the predictions from the csv files

PRED_DIR = os.path.join(ROOT_DIR, "CrabNet/model_predictions/")
print(PRED_DIR)
csv_files = os.listdir(PRED_DIR)

predicted_data = []
for _file in csv_files:
    with open(os.path.join(PRED_DIR, _file)) as train_file:
        csv_reader = csv.reader(train_file)
        
        row_count = 0
        for row in csv_reader:
            if row_count != 0:
#                 print(row)
                predicted_data.append({
                    "structure": row[1],
                    "predicted_e_ovac": row[3]
                })
            
            row_count += 1

            
print(predicted_data[:5])
print(len(predicted_data))
 

In [ ]:
print(min([x.get('predicted_e_ovac') for x in predicted_data]))
print(max([x.get('predicted_e_ovac') for x in predicted_data]))